<a href="https://colab.research.google.com/github/sarthakghosh21/Pizza_ML_RL/blob/main/5thAugDemo_Copy1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install featuretools
!pip install woodwork
!pip install composeml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.9/587.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.2/215.2 kB 11.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import featuretools as ft
import json
import warnings
warnings.filterwarnings('ignore')

In [4]:
def csv_to_dataframes(path):
  file=pd.read_csv(path)
  file['data'] = file['data'].apply(json.loads)
  data_df = file.join(pd.json_normalize(file['data'])).drop(columns=['data'])
  entities=data_df.entityname.unique()
  dataframes={}
  for entity in entities:
    df=pd.DataFrame()
    for index, row in data_df.iterrows():
      if(row['entityname']==entity):
        df = df._append(row, ignore_index = True)
    dataframes[entity]=df
  for key in dataframes:
    dataframes[key].dropna(inplace=True, axis=1)
  return dataframes


In [5]:
twin_data= csv_to_dataframes('twins_updated_corrected2 (1).csv')
twin_status = csv_to_dataframes('modified_twin_time_series_consistent_order_id.csv')

In [6]:
twin_data

{'order':      twin_id entityname    location  status relationship     customer  \
 0    order_1      order  front_desk  placed           {}   Customer_1   
 1    order_2      order  front_desk  placed           {}   Customer_2   
 2    order_3      order  front_desk  placed           {}   Customer_3   
 3    order_4      order  front_desk  placed           {}   Customer_4   
 4    order_5      order  front_desk  placed           {}   Customer_5   
 5    order_6      order  front_desk  placed           {}   Customer_6   
 6    order_7      order  front_desk  placed           {}   Customer_7   
 7    order_8      order  front_desk  placed           {}   Customer_7   
 8    order_9      order  front_desk  placed           {}   Customer_9   
 9   order_10      order  front_desk  placed           {}  Customer_10   
 10  order_11      order  front_desk  placed           {}  Customer_11   
 11  order_12      order  front_desk  placed           {}  Customer_12   
 12  order_13      order  fro

In [7]:
twin_status

{'order_1':              timestamp entityname     status           location  \
 0  2024-06-24 12:00:00    order_1     placed         front_desk   
 1  2024-06-24 12:13:00    order_1  processed            kitchen   
 2  2024-06-24 13:00:00    order_1   prepared            kitchen   
 3  2024-06-24 13:34:00    order_1      baked            kitchen   
 4  2024-06-24 14:01:00    order_1   packaged            kitchen   
 5  2024-06-24 14:33:00    order_1  delivered  customer_location   
 
   additional_info.customer additional_info.pizza_type  \
 0               Customer_1                     Veggie   
 1               Customer_1                     Veggie   
 2               Customer_1                     Veggie   
 3               Customer_1                     Veggie   
 4               Customer_1                     Veggie   
 5               Customer_1                     Veggie   
 
    additional_info.quantity additional_info.orderType  \
 0                       2.0                 

In [8]:
for key in twin_data.keys():
    print(key)

# for key in twin_status.keys():
#     print(key)

order
employee
machine
station
vehicle


In [9]:
key_val=input('Enter the main key value name: ')

Enter the main key value name: order


In [10]:
twin_status_data={}
for key in twin_data.keys():
  twin_status_data[key+'_status']=pd.DataFrame()

#issue 1
for key in twin_status.keys():
  for twin in twin_data.keys():
    temp=twin_data[twin]['twin_id']
    temp=np.array(temp)
    if key in temp:
      twin_status_data[twin+'_status']=pd.concat([twin_status_data[twin+'_status'],twin_status[key]],axis=0)
    else:
      continue

for key in twin_status_data.keys():
  twin_status_data[key].reset_index(inplace=True, drop=True)
  twin_status_data[key].reset_index(inplace=True)
  twin_status_data[key]['timestamp']=pd.to_datetime(twin_status_data[key]['timestamp'])

In [11]:
# import composeml as cp

# freq = input('Enter the desired frequency as 1d, 1m, 1y, etc. : ')

# def Sales(ds):
#     return ds['order_amount'].sum()

# sales_label_maker = cp.LabelMaker(
#     labeling_function = Sales,
#     time_index ='timestamp',
#     target_dataframe_index ='orderType',
#     window_size = freq,
# )

# sales = sales_label_maker.search(
#     twin_status_data["order_status"]=twin_status_data["order_status"].sort_values('timestamp'),
#     num_examples_per_instance=-1,
#     drop_empty=False,
#     verbose=False,
# )

# sales

In [12]:
#sales
daily_sales=twin_status_data['order_status'][twin_status_data['order_status']['status'].isin(['placed'])]

daily_sales=daily_sales.sort_values('timestamp')

daily_sales['timestamp']=pd.to_datetime(daily_sales['timestamp'])

daily_sales.set_index('timestamp',inplace=True)

daily_sales=daily_sales.asfreq('h')

# daily_sales = daily_sales.groupby([daily_sales['timestamp'].dt.date, daily_sales['timestamp'].dt.hour])['additional_info.order_amount'].sum().reset_index()

daily_sales

,index,entityname,status,location,additional_info.customer,additional_info.pizza_type,additional_info.quantity,additional_info.orderType,additional_info.order_amount
timestamp,,,,,,,,,
2024-06-24 12:00:00,0.0,order_1,placed,front_desk,Customer_1,Veggie,2.0,phone,20.0
2024-06-24 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-24 14:00:00,6.0,order_2,placed,front_desk,Customer_2,Veggie,3.0,online,30.0
2024-06-24 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-24 16:00:00,12.0,order_3,placed,front_desk,Customer_3,Pepperoni,4.0,phone,40.0
...,...,...,...,...,...,...,...,...,...
2024-06-27 16:00:00,102.0,order_18,placed,front_desk,Customer_20,BBQ Chicken,4.0,dine-in,40.0
2024-06-27 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-27 18:00:00,108.0,order_19,placed,front_desk,Customer_20,Veggie,5.0,online,50.0


In [13]:
# def Duration(ds):
#     return ds['timestamp_copy'].diff().dt.total_seconds().sum()/60
#     # .diff().dt.total_seconds().sum()/60

# # Duration(es['order_status'])
# lm3 = cp.LabelMaker(
#     labeling_function=Duration,
#     time_index='timestamp',
#     target_dataframe_index='entityname',
# )

# lt3 = lm3.search(
#     df=df1['order_status'].sort_values(['entityname','timestamp']),
#     num_examples_per_instance=-1,
#     drop_empty=False,
#     verbose=False,
# )

# lt3

In [14]:
twin_status_data['order_status'] = twin_status_data['order_status'].sort_values(['entityname','timestamp'])

process_duration = twin_status_data['order_status'][twin_status_data['order_status']['status'].isin(['placed', 'delivered'])][['timestamp','entityname','status']]

process_duration['total_duration'] = process_duration.groupby('entityname')['timestamp'].diff()

process_duration = process_duration[process_duration['status']=='delivered']

process_duration

,timestamp,entityname,status,total_duration
5,2024-06-24 14:33:00,order_1,delivered,0 days 02:33:00
59,2024-06-25 23:09:00,order_10,delivered,0 days 03:09:00
65,2024-06-26 14:04:00,order_11,delivered,0 days 02:04:00
71,2024-06-26 16:33:00,order_12,delivered,0 days 02:33:00
77,2024-06-26 18:31:00,order_13,delivered,0 days 02:31:00
83,2024-06-26 20:29:00,order_14,delivered,0 days 02:29:00
89,2024-06-26 22:24:00,order_15,delivered,0 days 02:24:00
95,2024-06-27 14:39:00,order_16,delivered,0 days 02:39:00
101,2024-06-27 15:58:00,order_17,delivered,0 days 01:58:00
107,2024-06-27 18:19:00,order_18,delivered,0 days 02:19:00


In [15]:
import composeml as cp
def Total_Quantity(ds):
    return ds['additional_info.quantity'].sum()

lm4 = cp.LabelMaker(
    labeling_function=Total_Quantity,
    time_index='timestamp',
    target_dataframe_index='additional_info.pizza_type',
    window_size='1d',
)

lt4 = lm4.search(
    df=twin_status_data['order_status'][twin_status_data['order_status']['status'].isin(['placed', 'delivered'])].sort_values('timestamp'),
    num_examples_per_instance=-1,
    drop_empty=False,
    verbose=False,
)

lt4
# def Number_of_orders(ds):
#     return ds['relationship.order_id'].count()

# lm5 = cp.LabelMaker(
#     labeling_function=Number_of_orders,
#     time_index='timestamp',
#     target_dataframe_index='entityname',
#     window_size='1d',
# )

# lt5 = lm5.search(
#     df=df1['employee_status'].sort_values('timestamp'),
#     num_examples_per_instance=-1,
#     drop_empty=False,
#     verbose=False,
# )

# lt5.rename(columns={'entityname': 'twin_id'},inplace=True)
# lt5

,additional_info.pizza_type,time,Total_Quantity
0,BBQ Chicken,2024-06-24 20:00:00,8.0
1,BBQ Chicken,2024-06-25 20:00:00,0.0
2,BBQ Chicken,2024-06-26 20:00:00,12.0
3,BBQ Chicken,2024-06-27 20:00:00,2.0
4,Hawaiian,2024-06-25 12:00:00,4.0
5,Hawaiian,2024-06-26 12:00:00,16.0
6,Margherita,2024-06-26 12:00:00,4.0
7,Margherita,2024-06-27 12:00:00,6.0
8,Pepperoni,2024-06-24 16:00:00,18.0
9,Pepperoni,2024-06-25 16:00:00,20.0


In [16]:
daily_orders= twin_status_data['order_status'][twin_status_data['order_status']['status'].isin(['placed'])][['timestamp','entityname']]

daily_orders = daily_orders.groupby(twin_status_data['order_status']['timestamp'].dt.date)['entityname'].count().reset_index()

daily_orders

,timestamp,entityname
0,2024-06-24,5
1,2024-06-25,5
2,2024-06-26,5
3,2024-06-27,5


In [17]:
es = ft.EntitySet(id = 'twin_data')
es

Entityset: twin_data
  DataFrames:
  Relationships:
    No relationships

In [18]:
#issue 2
from woodwork.logical_types import Categorical, NaturalLanguage

for i in twin_status_data.keys():
  if i.startswith(key_val):
    twin_status_data[i].ww.init(name=i, index='index',time_index="timestamp", logical_types={
   'entityname': Categorical,
   "location": Categorical,
   "status": Categorical,
  })
  else:
    twin_status_data[i].ww.init(name=i, index='index',time_index="timestamp", logical_types={
     'entityname': Categorical,
     "relationship."+key_val+"_id": Categorical
  })

for i in twin_data.keys():
  twin_data[i].ww.init(name=i, index='twin_id', logical_types={
   'twin_id': Categorical,
   "location":Categorical,
   "status":Categorical
})


In [19]:
twin_status_data['order_status'].ww.init(name='order_status', index='index',time_index="timestamp", logical_types={
   'entityname': Categorical,
   "location": Categorical,
   "status": Categorical,
})

In [20]:
twin_data['order'].ww.init(name='order', index='twin_id', logical_types={
   'twin_id': Categorical,
   'pizza_type': Categorical,
   'customer': NaturalLanguage,
   "orderType": Categorical,
   "location":Categorical,
   "status":Categorical
})

In [21]:
twin_status_data['employee_status'].ww.init(name='employee_status', index='index',time_index="timestamp",
  logical_types={
   'entityname': Categorical,
   "relationship.order_id": Categorical
}
                               )

In [22]:
twin_status_data['machine_status'].ww.init(name='machine_status', index='index',time_index="timestamp", logical_types={
   'entityname': Categorical,
   "relationship.order_id": Categorical
})

In [23]:
twin_data['employee'].ww.init(name='employee', index='twin_id', logical_types={
   'twin_id': Categorical,
   "entityname": Categorical,
   "location": Categorical,
   "status": Categorical,
   "name": NaturalLanguage

})

In [24]:
twin_data['machine'].ww.init(name='machine', index='twin_id', logical_types={
   'twin_id': Categorical,
   "entityname": Categorical,
   "location": Categorical,
   "status": Categorical,
   "model": NaturalLanguage

})


In [25]:
for i in twin_status_data.keys():
  es = es.add_dataframe(
      dataframe_name=i,
      dataframe=twin_status_data[i],
  )

for i in twin_data.keys():
  es = es.add_dataframe(
      dataframe_name=i,
      dataframe=twin_data[i],
  )

In [26]:
for i in twin_status_data.keys():
  for j in twin_data.keys():
    if i.startswith(j):
      es=es.add_relationship(j, "twin_id", i, "entityname")
    #issue 3
    if j==key_val and i.startswith(key_val)==False:
      es=es.add_relationship(j, "twin_id", i, "relationship."+key_val+"_id")

es

Entityset: twin_data
  DataFrames:
    order_status [Rows: 120, Columns: 10]
    employee_status [Rows: 40, Columns: 8]
    machine_status [Rows: 40, Columns: 8]
    station_status [Rows: 20, Columns: 6]
    vehicle_status [Rows: 20, Columns: 7]
    order [Rows: 20, Columns: 10]
    employee [Rows: 3, Columns: 6]
    machine [Rows: 3, Columns: 7]
    station [Rows: 1, Columns: 5]
    vehicle [Rows: 1, Columns: 6]
  Relationships:
    order_status.entityname -> order.twin_id
    employee_status.relationship.order_id -> order.twin_id
    employee_status.entityname -> employee.twin_id
    machine_status.relationship.order_id -> order.twin_id
    machine_status.entityname -> machine.twin_id
    station_status.relationship.order_id -> order.twin_id
    station_status.entityname -> station.twin_id
    vehicle_status.relationship.order_id -> order.twin_id
    vehicle_status.entityname -> vehicle.twin_id

In [27]:
from featuretools.selection import remove_highly_null_features, remove_single_value_features
user_df = input("Choose a Dataframe: ")
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name=user_df,
    max_depth=1,
    )
if user_df in twin_status_data:
  feature_matrix=pd.concat([feature_matrix,twin_status_data[user_df]['timestamp']],axis=1)

feature_matrix=remove_highly_null_features(feature_matrix)
feature_matrix=remove_single_value_features(feature_matrix)
feature_matrix

Choose a Dataframe: order_status


,entityname,status,location,additional_info.customer,additional_info.pizza_type,additional_info.quantity,additional_info.orderType,additional_info.order_amount,DAY(timestamp),WEEKDAY(timestamp),order.pizza_type,order.quantity,order.orderType,order.order_amount,timestamp
0,order_1,placed,front_desk,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 12:00:00
1,order_1,processed,kitchen,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 12:13:00
2,order_1,prepared,kitchen,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 13:00:00
3,order_1,baked,kitchen,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 13:34:00
6,order_2,placed,front_desk,Customer_2,Veggie,3.0,online,30.0,24,0,Veggie,3.0,online,30.0,2024-06-24 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,order_20,processed,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 20:33:00
116,order_20,prepared,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 20:57:00
117,order_20,baked,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 21:33:00
118,order_20,packaged,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 21:56:00


**AutoML**


In [28]:
freq='hour'
y=''
task=''

In [29]:
df = pd.DataFrame(feature_matrix)
df.to_csv('feature_matrix.csv', index=False)
pd.read_csv("feature_matrix.csv")

,entityname,status,location,additional_info.customer,additional_info.pizza_type,additional_info.quantity,additional_info.orderType,additional_info.order_amount,DAY(timestamp),WEEKDAY(timestamp),order.pizza_type,order.quantity,order.orderType,order.order_amount,timestamp
0,order_1,placed,front_desk,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 12:00:00
1,order_1,processed,kitchen,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 12:13:00
2,order_1,prepared,kitchen,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 13:00:00
3,order_1,baked,kitchen,Customer_1,Veggie,2.0,phone,20.0,24,0,Veggie,2.0,phone,20.0,2024-06-24 13:34:00
4,order_2,placed,front_desk,Customer_2,Veggie,3.0,online,30.0,24,0,Veggie,3.0,online,30.0,2024-06-24 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,order_20,processed,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 20:33:00
116,order_20,prepared,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 20:57:00
117,order_20,baked,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 21:33:00
118,order_20,packaged,kitchen,Customer_20,BBQ Chicken,1.0,dine-in,10.0,27,3,BBQ Chicken,1.0,dine-in,10.0,2024-06-27 21:56:00


In [30]:
y = input('Please select a column that you want to predict:\n'+str(feature_matrix.columns)+'\n')
y = feature_matrix[y]

X = feature_matrix.drop(y.name, axis=1)

Please select a column that you want to predict:
Index(['entityname', 'status', 'location', 'additional_info.customer',
       'additional_info.pizza_type', 'additional_info.quantity',
       'additional_info.orderType', 'additional_info.order_amount',
       'DAY(timestamp)', 'WEEKDAY(timestamp)', 'order.pizza_type',
       'order.quantity', 'order.orderType', 'order.order_amount', 'timestamp'],
      dtype='object')
status


In [31]:
#
selected_feature_matrix = X
#

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd

goal = input('Please select a task:\n' + 'Forecasting or Prediction\n')

if goal == 'Prediction':
    categorical_features = X.select_dtypes(include=['object']).columns.tolist()
    numerical_features = X.select_dtypes(include=['number']).columns.tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ]
    )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('feature_selection', RFE(estimator=RandomForestClassifier(), n_features_to_select=2))
    ])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    pipeline.fit(X_train, y_train)

    fitted_preprocessor = pipeline.named_steps['preprocessor']
    fitted_preprocessor.named_transformers_['cat'].fit(X_train[categorical_features])

    categorical_feature_names = fitted_preprocessor.named_transformers_['cat'].get_feature_names_out()
    feature_names = numerical_features + list(categorical_feature_names)
    rfe_selector = pipeline.named_steps['feature_selection']

    ranking = rfe_selector.ranking_
    selected_features = [name for name, rank in zip(feature_names, ranking) if rank == 1]

    # Save the relevant columns in a list called relevant_columns
    relevant_columns = selected_features

    print("Feature ranking:", ranking)
    print("Selected features:", selected_features)

    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model accuracy: {accuracy:.2f}")


Please select a task:
Forecasting or Prediction
Prediction
Feature ranking: [1 1 3 2]
Selected features: ['additional_info.quantity', 'additional_info.order_amount']
Model accuracy: 0.03


In [33]:
def determine_task_type(y, feature_matrix):
    if goal == 'Prediction':
        if y.dtype == "int64":
            return 'regression', feature_matrix
        elif y.dtype == 'category':
            return 'classification', feature_matrix
        else:
            return 'Error: y selected is invalid', feature_matrix

    elif goal == "Forecasting":
        feature_matrix['timestamp'] = pd.to_datetime(feature_matrix['timestamp'])
        feature_matrix = feature_matrix.sort_values('timestamp')
        feature_matrix.set_index('timestamp', inplace=True)
        feature_matrix = feature_matrix[y.name].resample('H').sum()
        feature_matrix = pd.DataFrame(feature_matrix)
        feature_matrix.reset_index(inplace=True)
        return 'ts_forecast', feature_matrix

    else:
        return 'Error: Incorrect goal selected', feature_matrix

task, feature_matrix = determine_task_type(y, feature_matrix)
print(task)
print(feature_matrix)

classification
    entityname     status           location additional_info.customer  \
0      order_1     placed         front_desk               Customer_1   
1      order_1  processed            kitchen               Customer_1   
2      order_1   prepared            kitchen               Customer_1   
3      order_1      baked            kitchen               Customer_1   
6      order_2     placed         front_desk               Customer_2   
..         ...        ...                ...                      ...   
115   order_20  processed            kitchen              Customer_20   
116   order_20   prepared            kitchen              Customer_20   
117   order_20      baked            kitchen              Customer_20   
118   order_20   packaged            kitchen              Customer_20   
119   order_20  delivered  customer_location              Customer_20   

    additional_info.pizza_type  additional_info.quantity  \
0                       Veggie                  

In [41]:
def determine_task_type(y, feature_matrix):
    if goal == 'Prediction':
        if y.dtype == "int64":
            return 'regression', feature_matrix
        elif y.dtype == 'category':
            return 'classification', feature_matrix
        else:
            return 'Error: y selected is invalid', feature_matrix

    elif goal == "Forecasting":
        feature_matrix['timestamp'] = pd.to_datetime(feature_matrix['timestamp'])
        feature_matrix = feature_matrix.sort_values('timestamp')
        feature_matrix.set_index('timestamp', inplace=True)
        feature_matrix = feature_matrix[y.name].resample('H').sum()
        feature_matrix = pd.DataFrame(feature_matrix)
        feature_matrix.reset_index(inplace=True)
        return 'ts_forecast', feature_matrix

    else:
        return 'Error: Incorrect goal selected', feature_matrix

task, feature_matrix = determine_task_type(y, feature_matrix)
print(task)
print(feature_matrix)

from sklearn.model_selection import train_test_split

selected_columns_set = set()

if task == 'ts_forecast':
    feature_matrix.sort_values(by='timestamp', inplace=True)
    X = feature_matrix['timestamp']
    y = feature_matrix[y.name]
    size = int(len(feature_matrix) * 0.8)
    X_train = np.array(X[:size])
    X_test = np.array(X[size:])
    y_train = np.array(y[:size])
    y_test = np.array(y[size:])

elif task in ['regression', 'classification']:
    X = feature_matrix.drop(columns=[y.name], axis=1, inplace=False)
    print("Relevant columns selected by RFE:")
    for col in relevant_columns:
        print(col)

    additional_columns = []
    add_more = input("Do you want to add additional columns to the RFE list? (yes/no): ")
    if add_more.lower() == 'yes':
        unique_columns = list(set(X.columns))
        print("Select columns on which you want to train your model:")
        print("Input 0 to exclude column")
        print("Input 1 to include column\n")
        for i in X.columns:
            if i not in unique_columns:
                x = int(input(f"{i}: "))
                if x == 0:
                    X.drop(columns=[i], axis=1, inplace=True)
                elif x == 1:
                    pass
    else:
        print("Training the model on the RFE relevant columns")
        selected_columns_set = set(relevant_columns)
        X = X[list(selected_columns_set)]

    if add_more.lower() == 'yes':
        for i in X.columns:
            if i not in selected_columns_set:
                x = int(input(f"{i}: "))
                if x == 0:
                    X.drop(columns=[i], axis=1, inplace=True)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Define automl_settings based on the task
if task == 'regression':
    automl_settings = {
        "metric": "rmse",
    }
elif task == 'classification':
    automl_settings = {
        "metric": "accuracy",
    }
elif task == 'ts_forecast':
    automl_settings = {
        "metric": "rmse",
        "period": 7,
    }
else:
    print('Error')

print(y.name)

classification
    entityname     status           location additional_info.customer  \
0      order_1     placed         front_desk               Customer_1   
1      order_1  processed            kitchen               Customer_1   
2      order_1   prepared            kitchen               Customer_1   
3      order_1      baked            kitchen               Customer_1   
6      order_2     placed         front_desk               Customer_2   
..         ...        ...                ...                      ...   
115   order_20  processed            kitchen              Customer_20   
116   order_20   prepared            kitchen              Customer_20   
117   order_20      baked            kitchen              Customer_20   
118   order_20   packaged            kitchen              Customer_20   
119   order_20  delivered  customer_location              Customer_20   

    additional_info.pizza_type  additional_info.quantity  \
0                       Veggie                  

In [40]:
!pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 7.5 MB/s eta 0:00:00


In [42]:
from flaml import AutoML
import joblib

automl = AutoML()
automl.fit(
    X_train=X_train,
    y_train=y_train,
    task=task,
    time_budget=15,
    **automl_settings
)

[flaml.automl.logger: 08-22 15:39:37] {1680} INFO - task = classification
[flaml.automl.logger: 08-22 15:39:37] {1691} INFO - Evaluation method: cv


INFO:flaml.automl.task.generic_task:class 0 augmented from 17 to 34
INFO:flaml.automl.task.generic_task:class 1 augmented from 13 to 26
INFO:flaml.automl.task.generic_task:class 2 augmented from 13 to 26
INFO:flaml.automl.task.generic_task:class 3 augmented from 10 to 20
INFO:flaml.automl.task.generic_task:class 4 augmented from 14 to 28
INFO:flaml.automl.task.generic_task:class 5 augmented from 13 to 26


[flaml.automl.logger: 08-22 15:39:37] {1789} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 08-22 15:39:37] {1901} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 08-22 15:39:37] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 08-22 15:39:39] {2345} INFO - Estimated sufficient time budget=18551s. Estimated necessary time budget=428s.
[flaml.automl.logger: 08-22 15:39:39] {2392} INFO -  at 2.0s,	estimator lgbm's best error=0.8000,	best estimator lgbm's best error=0.8000
[flaml.automl.logger: 08-22 15:39:39] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 08-22 15:39:40] {2392} INFO -  at 2.9s,	estimator lgbm's best error=0.7875,	best estimator lgbm's best error=0.7875
[flaml.automl.logger: 08-22 15:39:40] {2219} INFO - iteration 2, current learner lgbm
[flaml.automl.logger: 08-22 15:39:41] {2392} INFO -  at 3.6s,	estimator lgbm's best error=0.78

In [43]:
# import pandas as pd

# if task == "ts_forecast":
#     input_features = X_train

#     # Save the column names to a CSV file
#     input_features_df = pd.DataFrame(input_features, columns=['Feature'])
#     input_features_df.to_csv('input_features_ts.csv', index=False)

# elif task == "classification":
#     input_features = X_train.columns

#     # Save the column names to a CSV file
#     input_features_df = pd.DataFrame(input_features, columns=['Feature'])
#     input_features_df.to_csv('input_features_pred.csv', index=False)

#     # Print the column names
# print(input_features)


In [44]:
X_train

,additional_info.order_amount,additional_info.quantity
33,20.0,2.0
25,10.0,1.0
69,30.0,3.0
28,10.0,1.0
98,30.0,3.0
...,...,...
106,40.0,4.0
14,40.0,4.0
92,20.0,2.0
51,50.0,5.0


In [45]:
from sklearn.metrics import mean_squared_error, accuracy_score
train_pred=automl.predict(X_train)
test_pred=automl.predict(X_test)
if(task=='regression' or task=='ts_forecast'):
  print(f'Train RMSE: {np.sqrt(mean_squared_error(y_train,train_pred))}')
  print(f'Test RMSE: {np.sqrt(mean_squared_error(y_test,test_pred))}')
elif task=='classification':
  print(f'Train Accuracy: {accuracy_score(y_train,train_pred)}')
  print(f'Test Accuracy: {accuracy_score(y_test,test_pred)}')
else:
  print('Error')

Train Accuracy: 0.2125
Test Accuracy: 0.075


In [46]:
import flaml
import joblib

best_estimator_name = automl.best_estimator
starting_points = automl.best_config_per_estimator

joblib.dump(automl, 'automl_model.pkl')
loaded_automl = joblib.load('automl_model.pkl')

print("Best Estimator Name:", best_estimator_name)

new_automl = flaml.AutoML()

if task == 'ts_forecast':
    try:
        new_automl.fit(
            X_train=X_train,
            y_train=y_train,
            task="ts_forecast",
            period=7,
            starting_points=starting_points,
            estimator_list=[best_estimator_name],
            time_budget=15,
            early_stop={'max_iter_without_improvement': 50},
            ensemble=True
        )
        joblib.dump(new_automl, 'tuned_automl_.pkl')

    except KeyError as e:
        print(f"KeyError: {e}")

else:
    try:
        new_automl.fit(
            X_train=X_train,
            y_train=y_train,
            starting_points=starting_points,
            estimator_list=[best_estimator_name],
            time_budget=15,
            early_stop={'max_iter_without_improvement': 50},
            ensemble=True
        )
        joblib.dump(new_automl, 'tuned_automl_model.pkl')

        # FINAL RUN OF THE SAVED MODEL
        print("Available columns:", selected_feature_matrix.columns.tolist())

        y_column = input('Please select a column that you want to predict:\n')

        y = selected_feature_matrix[y_column]
        X = selected_feature_matrix.drop(columns=[y_column])
        automl_model = joblib.load('automl_model.pkl')

        predictions = automl_model.predict(X)
        print("Predictions:", predictions)

    except KeyError as e:
        print(f"KeyError: {e}")


Best Estimator Name: lgbm
[flaml.automl.logger: 08-22 15:39:53] {1680} INFO - task = classification
[flaml.automl.logger: 08-22 15:39:53] {1691} INFO - Evaluation method: cv


INFO:flaml.automl.task.generic_task:class 0 augmented from 17 to 34
INFO:flaml.automl.task.generic_task:class 1 augmented from 13 to 26
INFO:flaml.automl.task.generic_task:class 2 augmented from 13 to 26
INFO:flaml.automl.task.generic_task:class 3 augmented from 10 to 20
INFO:flaml.automl.task.generic_task:class 4 augmented from 14 to 28
INFO:flaml.automl.task.generic_task:class 5 augmented from 13 to 26


[flaml.automl.logger: 08-22 15:39:53] {1789} INFO - Minimizing error metric: log_loss
[flaml.automl.logger: 08-22 15:39:53] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 08-22 15:39:53] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 08-22 15:39:54] {2345} INFO - Estimated sufficient time budget=6184s. Estimated necessary time budget=6s.
[flaml.automl.logger: 08-22 15:39:54] {2392} INFO -  at 0.7s,	estimator lgbm's best error=1.7843,	best estimator lgbm's best error=1.7843
[flaml.automl.logger: 08-22 15:39:54] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 08-22 15:39:54] {2392} INFO -  at 1.1s,	estimator lgbm's best error=1.7820,	best estimator lgbm's best error=1.7820
[flaml.automl.logger: 08-22 15:39:54] {2219} INFO - iteration 2, current learner lgbm
[flaml.automl.logger: 08-22 15:39:55] {2392} INFO -  at 1.6s,	estimator lgbm's best error=1.7820,	best estimator lgbm's best error=1.7820
[flaml.automl.logg

In [47]:
task

'classification'

In [ ]:
# """X_train gets saved for Pred"""
# import pandas as pd

# task = 'ts_forecast'
# if task == 'ts_forecast':

#     timestamp_column = feature_matrix[['timestamp']]
#     timestamp_column.to_csv('input_features.csv', index=False)

# else:
#     print("Task is not 'ts_forecast'; skipping CSV save.")

# if task == "classification":
#     X_train.to_csv('Prediction.csv', index=False)
#     print(X_train.columns)

In [48]:
from datetime import datetime, timedelta

def generate_next_datetimes(timestamps, n):
    """
    Generate next n datetime values distributed at the same interval as timestamps.

    Args:
    - timestamps (list of datetime): Sorted list of datetime objects.
    - n (int): Number of datetime values to generate.

    Returns:
    - list of datetime: List containing the next n datetime values.
    """
    if len(timestamps) < 2:
        raise ValueError("List of timestamps must have at least two elements.")

    interval = timestamps[-1] - timestamps[-2]
    next_datetimes = [timestamps[-1] + (i + 1) * interval for i in range(n)]

    return next_datetimes


In [ ]:
if goal=='Forecasting':
  period=int(input('How many future values do you want to predict? '))

  X_input=generate_next_datetimes(X_test,period)
  X_input=np.array(X_input)

  pred=automl.predict(X_input)
  y_output = pd.DataFrame({'Timestamp': X_input, 'Forecast': pred})
  print(y_output)

In [ ]:
import plotly.graph_objs as go
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

def plot_results(goal, X_test, X_train, y_test, y_train, X_input, pred):
    if goal == 'Forecasting':
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=X_test, y=y_test, mode='lines', name='Actual'))
        fig.add_trace(go.Scatter(x=X_input, y=pred, mode='lines', name='Forecast', line=dict(dash='dash')))
        fig.update_layout(
            xaxis_title='Timestamp',
            yaxis_title='Value',
            title='Forecasting Results',
            legend=dict(x=0, y=1, traceorder='normal'),
            showlegend=True,
            xaxis=dict(showgrid=True),
            yaxis=dict(showgrid=True),
        )
        fig.show()

    elif goal == 'Prediction':
        # for x input
        widgets_dict = {}
        for col in X_input.columns:
            if X_input[col].dtype == 'int64':
                widgets_dict[col] = widgets.IntText(value=int(X_input[col][0]), description=f'{col}:')
            elif X_input[col].dtype == 'float64':
                widgets_dict[col] = widgets.FloatText(value=float(X_input[col][0]), description=f'{col}:')
            else:
                widgets_dict[col] = widgets.Text(value=str(X_input[col][0]), description=f'{col}:')


        display(*widgets_dict.values())

        # Function to update the prediction
        def update_prediction(b):
            for col in X_input.columns:
                if X_input[col].dtype == 'int64':
                    X_input.loc[0, col] = int(widgets_dict[col].value)
                elif X_input[col].dtype == 'float64':
                    X_input.loc[0, col] = float(widgets_dict[col].value)
                else:
                    X_input.loc[0, col] = widgets_dict[col].value

            new_pred = automl.predict(X_input)
            X_input['prediction'] = new_pred
            display(X_input)




    else:
        print('Error: Invalid goal')

if goal == 'Forecasting':
    period = int(input('How many future values do you want to predict? '))

    X_input = generate_next_datetimes(X_test, period)
    X_input = np.array(X_input)

    pred = automl.predict(X_input)
    y_output = pd.DataFrame({'Timestamp': X_input, 'Forecast': pred})
    print(y_output)

    # call plot results for forecasting
    plot_results(goal, X_test, X_train, y_test, y_train, X_input, pred)

elif goal == 'Prediction':
    X_input = pd.DataFrame()

    for i in X_train.columns:
        if X_train[i].dtype == 'int64':
            val = int(input(f'Enter {i}: '))
        elif X_train[i].dtype == 'float64':
            val = float(input(f'Enter {i}: '))
        else:
            val = input(f'Enter {i}: ')
        X_input.loc[0, i] = val

    for i in X_train.columns:
        if X_train[i].dtype == '<M8[ns]':
            X_input[i] = pd.to_datetime(X_input[i])

    pred = automl.predict(X_input)
    X_input['prediction'] = pred
    print(X_input)

    # call plot results for prediction
    plot_results(goal, X_test, X_train, y_test, y_train, X_input, pred)

else:
    print('Error')
